In [ ]:
from huggingface_hub import login

# Prompt the user for their Hugging Face token
huggingface_token = input("Enter your Hugging Face token: ")

# Log in to Hugging Face
login(token=huggingface_token)

print("Successfully logged in to Hugging Face!")


: 

In [2]:
# import torch 
# for i in range(torch.cuda.device_count()):     
#     print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

In [6]:
from datasets import load_dataset

# Load the dataset from Hugging Face Hub
dataset_name = "akhilfau/physics_decontaminated_2"
dataset = load_dataset(dataset_name, split="train")  # Adjust the split if needed (e.g., "test" or "validation")

# Print a sample record
print("Sample record from the dataset:")
print(dataset[0])  # Prints the first record from the dataset


Sample record from the dataset:
{'role_1': 'Physicist_RoleType.ASSISTANT', 'topic;': 'Quantum mechanics', 'sub_topic': 'The Schrödinger equation and its solutions', 'message_1': 'What is the probability of finding a particle with a given energy in a one-dimensional infinite square well potential when the potential width is 2 nm and the particle has a mass of 5x10^-26 kg? Use the Schrödinger equation to solve for the allowed energy states and their wave functions.', 'message_2': 'To find the probability of finding a particle with a given energy in a one-dimensional infinite square well potential, we first need to solve the Schrödinger equation for the allowed energy states and their wave functions.\n\nThe time-independent Schrödinger equation for a one-dimensional infinite square well potential is given by:\n\n- (ħ^2 / 2m) * (d^2ψ(x) / dx^2) = E * ψ(x)\n\nwhere ħ is the reduced Planck constant (1.0545718 × 10^-34 Js), m is the mass of the particle (5 × 10^-26 kg), E is the energy of the

In [9]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch

# Step 1: Load the Dataset
dataset = load_dataset("akhilfau/physics_decontaminated_2", split="train")

# Step 2: Load the Pretrained Model and Tokenizer
model_name = "HuggingFaceTB/SmolLM2-360M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define the padding token if not already set
tokenizer.pad_token = tokenizer.eos_token or tokenizer.bos_token or "[PAD]"

# Step 3: Configure LoRA with PEFT
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    task_type="CAUSAL_LM",  # Task type for causal language modeling
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Print trainable parameters to confirm LoRA is applied
model.print_trainable_parameters()

# Step 4: Preprocess the Dataset
def preprocess_function(examples):
    # Concatenate the problem and solution for causal LM
    inputs = [f"Problem: {problem}\nSolution: {solution}" for problem, solution in zip(examples["message_1"], examples["message_2"])]
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    
    # Labels are the same as input_ids for causal LM
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Split into train and validation sets
# train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
# train_dataset = train_test_split["train"]
# eval_dataset = train_test_split["test"]

print("length of the data: ",tokenized_dataset.shape[0])

# Step 5: Define Training Arguments
# training_args = TrainingArguments(
#     output_dir="./fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics",
#     evaluation_strategy="epoch",
#     logging_steps=100,
#     save_steps=500,
#     learning_rate=5e-4,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     save_total_limit=2,
#     push_to_hub=False,  # Set to True if you want to push to Hugging Face Hub
#     logging_dir="./logs",
#     bf16=False,  # Disable BFloat16
#     fp16=False,  # Disable FP16
# )

# training_args = TrainingArguments(
#     output_dir="./fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics",
#     evaluation_strategy="epoch",
#     logging_steps=100,
#     save_steps=500,
#     learning_rate=5e-4,  # Reduce the learning rate
#     lr_scheduler_type="cosine",  # Use a more adaptive scheduler
#     per_device_train_batch_size=4,  # Increase if memory allows
#     per_device_eval_batch_size=4,
#     num_train_epochs=8,  # Train for more epochs
#     weight_decay=0.1,  # Regularization
#     save_total_limit=2,
#     logging_dir="./logs",
#     push_to_hub=False,
#     #gradient_checkpointing=True,  # Reduce memory usage if needed
#     bf16=False,  # Disable BFloat16
#     fp16=False,  # Disable FP16
# )

#Step5:
# Estimate total training steps
use_bf16 = torch.cuda.is_bf16_supported()
dataset_size = tokenized_dataset.shape[0] # Adjust based on dataset size
batch_size = 4  # Adjust based on available GPU memory
grad_accum_steps = 4  # Simulate larger batch without using more memory
total_steps = (dataset_size // (batch_size * grad_accum_steps)) * 3  # 3 epochs

# Training Arguments
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=grad_accum_steps,
    warmup_steps=100,  # More warmup steps for stability
    max_steps=total_steps,  # Train across full dataset
    learning_rate=2e-4,
    fp16=not use_bf16,  # Use FP16 if bf16 is not available
    bf16=use_bf16,  # Use BF16 if supported
    logging_steps=100,  # Reduce logging frequency for better efficiency
    save_steps=1000,  # Save every 1000 steps
    save_total_limit=2,  # Keep last 2 checkpoints
    optim="adamw_8bit",  # Memory-efficient optimizer
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="wandb",  # Log to W&B
)
# Step 6: Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
    # eval_dataset=eval_dataset,
)

# Step 7: Train the Model
trainer.train()



trainable params: 1,638,400 || all params: 363,459,520 || trainable%: 0.4508
length of the data:  20000


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/home/avallala2023/anaconda3/envs/Thesis/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
100,0.409000
200,0.231700
300,0.222700
400,0.220500
500,0.216900
600,0.216400
700,0.215000
800,0.213100
900,0.211100
1000,0.210700


/home/avallala2023/anaconda3/envs/Thesis/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/avallala2023/anaconda3/envs/Thesis/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/avallala2023/anaconda3/envs/Thesis/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=3750, training_loss=0.2113407958984375, metrics={'train_runtime': 8338.1042, 'train_samples_per_second': 28.784, 'train_steps_per_second': 0.45, 'total_flos': 2.328401992482816e+17, 'train_loss': 0.2113407958984375, 'epoch': 11.9824})

In [10]:
# Step 8: Save the Model and Tokenizer
save_dir = "./fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('./fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2/tokenizer_config.json',
 './fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2/special_tokens_map.json',
 './fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2/vocab.json',
 './fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2/merges.txt',
 './fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2/added_tokens.json',
 './fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2/tokenizer.json')

In [11]:
#Testing locally

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the path to the locally saved model
local_model_path = "./fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForCausalLM.from_pretrained(local_model_path)

# Test with a sample input
#input_text = "What is the Schrödinger equation?"
input_text = "A car accelerates uniformly from rest to a speed of 30 m/s in 10 seconds. What is the acceleration of the car?Options:A) 3 m/s²B) 2.5 m/s²C) 5 m/s²D) 4 m/s²"
inputs = tokenizer(input_text, return_tensors="pt")

# Generate a response
output = model.generate(**inputs, max_length=50)
print(tokenizer.decode(output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


ValueError: Input length of input_ids is 66, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [13]:
from huggingface_hub import create_repo, upload_folder, login
# Create the repository (if it doesn't exist)
repo_name="akhilfau/fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2"
create_repo(repo_id=repo_name, repo_type="model", exist_ok=True)

RepoUrl('https://huggingface.co/akhilfau/fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2', endpoint='https://huggingface.co', repo_type='model', repo_id='akhilfau/fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2')

In [16]:

# Push to Hugging Face Hub
trainer.push_to_hub(commit_message="Fine-tuned smolLM2-360M with LoRA on camel-ai/physics_2")

from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="./fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2",
    repo_id="akhilfau/fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2"
)


training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.57M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/6.57M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/akhilfau/fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2/commit/dd94f79c05c8ae171de188cd8a802650396062b1', commit_message='Upload folder using huggingface_hub', commit_description='', oid='dd94f79c05c8ae171de188cd8a802650396062b1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/akhilfau/fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2', endpoint='https://huggingface.co', repo_type='model', repo_id='akhilfau/fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2'), pr_revision=None, pr_num=None)

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the Hugging Face model path (update with the correct repository path)
model_path = "akhilfau/fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics_2"

# Load the model and tokenizer from the Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Test with a sample input
#input_text = "What is the Schrödinger equation?"
input_text = "A car accelerates uniformly from rest to a speed of 30 m/s in 10 seconds. What is the acceleration of the car?Options:A) 3 m/s²B) 2.5 m/s²C) 5 m/s²D) 4 m/s²"

inputs = tokenizer(input_text, return_tensors="pt")

# Generate a response
output = model.generate(**inputs, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


A car accelerates uniformly from rest to a speed of 30 m/s in 10 seconds. What is the acceleration of the car?Options:A) 3 m/s²B) 2.5 m/s²C) 5 m/s²D) 4 m/s²

asked by Anonymous on March 1, 2018
10. ## Physics

A car accelerates uniformly from rest to a speed of 30 m/s in 10 seconds. What is the acceleration of the car? Options:A) 3 m/s²B) 2.5 m/s²C) 5 m/s²D) 4 m/s²

asked by Anonymous on March 1, 2018
11. ## Physics

A car accelerates uniformly from rest to a speed of 30 m/s in 10 seconds. What is the acceleration of the car? Options:A) 3 m/s²B) 2.5 m/s²C) 5 m/s²D) 4 m/s²

asked by Anonymous on March 1, 2018
12. ## Physics

A car accelerates uniformly from rest to a speed of 30 m/s in 10 seconds. What is the acceleration of the car? Options:A) 3 m/s²B) 2.5 m/s²C) 5 m/s²D) 4 m/s²

asked by Anonymous on March 1, 2018
13. ## Physics

A car accelerates uniformly from rest to a speed of 30 m/s in 10 seconds. What is the acceleration of the car? Options:A) 3 m/s²B) 2.5 m/s²C) 5 m/s²D) 4 m/s²

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the Hugging Face model path (update with the correct repository path)
model_path = "akhilfau/Instruction_fine_tuned_on_camel_ai_physics"

# Load the model and tokenizer from the Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Test with a sample input
#input_text = "What is the Schrödinger equation?"
input_text = "A car accelerates uniformly from rest to a speed of 30 m/s in 10 seconds. What is the acceleration of the car?Options:A) 3 m/s²B) 2.5 m/s²C) 5 m/s²D) 4 m/s²"

inputs = tokenizer(input_text, return_tensors="pt")

# Generate a response
output = model.generate(**inputs, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))


A car accelerates uniformly from rest to a speed of 30 m/s in 10 seconds. What is the acceleration of the car?Options:A) 3 m/s²B) 2.5 m/s²C) 5 m/s²D) 4 m/s²E) 6 m/s²

ANSWER: A) 3 m/s²

A car accelerates from rest to a speed of 30 m/s in 10 seconds. To find the acceleration, we need to determine the distance covered and the change in speed.

Distance (d) = total speed × time

Change in speed (Δs) = final speed - initial speed

First, we need to find the final speed (v) of the car:

v = final speed = 30 m/s

Now, we can find the change in distance (Δd) by adding the initial distance (d) to the final distance (v):

Δd = d + v = 0 + 30 m/s = 30 m/s

Now we can find the change in time (Δt) by dividing the change in distance by the time it took to accelerate from rest to 30 m/s:

Δt = Δd / Δs = 30 m/s / (30 m/s) = 1 second

Now we can find the acceleration (a) using the formula:

a = Δt / Δs

a = 1 s / 30 m = 1/30 m/s²

So, the acceleration of the car is 1/30 m/s² or 3/30 m/s² (1/10 m/s² or

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the Hugging Face model path (update with the correct repository path)
model_path = "akhilfau/Instruction_fine_tuned_on_camel_ai_physics"

# Load the model and tokenizer from the Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Test with a sample input
#input_text = "What is the Schrödinger equation?"
input_text = "A ball is thrown vertically upward with an initial velocity of 20 m/s. Ignoring air resistance, how long will it take for the ball to reach its highest point?"

inputs = tokenizer(input_text, return_tensors="pt")

# Generate a response
output = model.generate(**inputs, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))


A ball is thrown vertically upward with an initial velocity of 20 m/s. Ignoring air resistance, how long will it take for the ball to reach its highest point? Assume the Earth's gravitational acceleration is 9.81 m/s^2. Calculate the time duration of the thrown ball's motion using the formula for vertical motion-

t = v*sin(θ) / g

where t is the time duration, v is the initial velocity, θ is the angle of projection, and g is the acceleration due to gravity.

Given:
v = 20 m/s
θ = 0° (thrown directly upwards)
g = 9.81 m/s² (Earth's gravitational acceleration)

First, we need to find the angle of projection (θ) using the initial velocity and distance traveled (x). We can use the formula:

x = v*sin(θ)

x = 20 m/s * sin(0°)
x = 20 m/s * 0 m/s
x = 0 m

Since x = 0, we cannot have a sine value of 0. This means that the thrown ball will not reach its highest point and will continue to fall until it reaches the ground it was thrown from. Therefore, the time duration of the thrown ball's moti

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the Hugging Face model path (update with the correct repository path)
model_path = "akhilfau/Instruction_fine_tuned_on_camel_ai_physics"

# Load the model and tokenizer from the Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Test with a sample input
#input_text = "What is the Schrödinger equation?"
input_text = "A 5 kg box is being pulled across a rough surface with a force of 40 N. If the coefficient of kinetic friction between the box and the surface is 0.3, what is the acceleration of the box?"

inputs = tokenizer(input_text, return_tensors="pt")

# Generate a response
output = model.generate(**inputs, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))


A 5 kg box is being pulled across a rough surface with a force of 40 N. If the coefficient of kinetic friction between the box and the surface is 0.3, what is the acceleration of the box?

(A) 0 m/s² (B) 3 m/s² (C) 0.3 m/s² (D) 9 m/s² (E) Not sure

(F) Please help solve this physics problem for a physics project – If you have the answer, I would love to see it! If not, don’t worry about the answer itself, just want to know how you solved it. Thanks in advance for your time and help.

1. A. (F)

(F) = 40 N

(F) = 0.3 * 40 N

(F) = 12 N

(F) = 0.3 * 12 N

(F) = 3 N

(F) = 0.3 * 3 N

(F) = 9 N

(F) = 9 m/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 9 cm/s²

(F) = 


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the Hugging Face model path (update with the correct repository path)
model_path = "HuggingFaceTB/SmolLM2-360M-Instruct"

# Load the model and tokenizer from the Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Test with a sample input
#input_text = "What is the Schrödinger equation?"
input_text = "A 5 kg box is being pulled across a rough surface with a force of 40 N. If the coefficient of kinetic friction between the box and the surface is 0.3, what is the acceleration of the box?"

inputs = tokenizer(input_text, return_tensors="pt")

# Generate a response
output = model.generate(**inputs, max_length=512)
print(tokenizer.decode(output[0], skip_special_tokens=True))


tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

A 5 kg box is being pulled across a rough surface with a force of 40 N. If the coefficient of kinetic friction between the box and the surface is 0.3, what is the acceleration of the box?


In [18]:
# import torch

# # Clear cache
# torch.cuda.empty_cache()

# # Reset memory allocations and free up GPU memory
# torch.cuda.memory_summary(device=None, abbreviated=False)


In [19]:
MODEL = "HuggingFaceTB/SmolLM2-360M"
OUTPUT_DIR = "/content/output"
 
!lighteval accelerate \
    --model_args "pretrained=$MODEL" \
    --tasks "leaderboard|mmlu:college_physics|0|0" \
    --override_batch_size 16 \
    --output_dir $OUTPUT_DIR

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/bin/bash: line 1: lighteval: command not found


In [ ]:
MODEL = "akhilfau/fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics"
OUTPUT_DIR = "/content/output"
 
!lighteval accelerate \
    --model_args "pretrained=$MODEL" \
    --tasks "leaderboard|mmlu:college_physics|0|0" \
    --override_batch_size 16 \
    --output_dir $OUTPUT_DIR

In [ ]:
MODEL = "HuggingFaceTB/SmolLM2-360M"
OUTPUT_DIR = "/content/output"
!lighteval accelerate \
    --model_args "pretrained=$MODEL" \
    --tasks "leaderboard|mmlu:conceptual_physics|0|0" \
    --override_batch_size 16 \
    --output_dir "$OUTPUT_DIR"


In [ ]:
MODEL = "akhilfau/fine-tuned-smolLM2-360M-with-LoRA-on-camel-ai-physics"
OUTPUT_DIR = "/content/output"
!lighteval accelerate \
    --model_args "pretrained=$MODEL" \
    --tasks "leaderboard|mmlu:conceptual_physics|0|0" \
    --override_batch_size 16 \
    --output_dir "$OUTPUT_DIR"
